In [1]:
import os
import sys
import inspect

sys.path.insert(1, os.path.join(sys.path[0], '..'))

import fatapi
from fatapi.data import Data
from fatapi.model import BlackBox, Model
from fatapi.methods import FACEMethod
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=100, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

clf.predict_proba(X_test[:1])
clf.predict(X_test[:5, :])
clf.score(X_test, y_test)
print(X_text)
print(y_test)
data_X = Data(dataset=X_test, dtype="data")
data_y = Data(dataset=y_test, dtype="target")
blackb = BlackBox(clf)
face_model = Model(data_X, data_y, blackb)
face_method = FACEMethod()

None
None
0000000000 None


TypeError: 'NoneType' object is not iterable